In [1]:
import pandas as pd
from pathlib import Path
from function import load_data

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
files_list_series = pd.Series([i for i in Path("./nav_data").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}])

In [3]:
fund_info = load_data("产品代码.xlsx")
fund_info["产品代码"] = fund_info["产品代码"].astype(str)

In [4]:
nav_dfs = pd.DataFrame()
for row in fund_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[0] in x.stem)]
    nav_df = load_data(nav_df_path.iloc[0])
    nav_df = nav_df.rename(columns={'基金id': '产品名称', '累计单位净值': '复权净值'})
    nav_df["产品代码"] = row[0]
    nav_df = nav_df[["产品代码","产品名称","日期","单位净值","累计净值","复权净值"]]
    nav_dfs = pd.concat([nav_dfs, nav_df], ignore_index=True)

In [7]:
nav_dfs.to_csv("nav_dfs.csv",index=False)

In [8]:
nav_dfs

,产品代码,产品名称,日期,单位净值,累计净值,复权净值
0,SGV993,聚鸣涌金8号A期,2020-01-22,1.0000,1.0000,1.000000
1,SGV993,聚鸣涌金8号A期,2020-02-05,1.0000,1.0000,1.000000
2,SGV993,聚鸣涌金8号A期,2020-02-06,1.0000,1.0000,1.000000
3,SGV993,聚鸣涌金8号A期,2020-02-07,1.0000,1.0000,1.000000
4,SGV993,聚鸣涌金8号A期,2020-02-10,1.0000,1.0000,1.000000
...,...,...,...,...,...,...
20487,SXA265,玖鹏宏远3号,2025-02-05,0.9712,1.0965,1.092891
20488,SXA265,玖鹏宏远3号,2025-02-06,0.9561,1.0814,1.075899
20489,SXA265,玖鹏宏远3号,2025-02-11,1.0120,1.1373,1.138804
20490,SXA265,玖鹏宏远3号,2025-02-18,1.0401,1.1654,1.170425


In [ ]:
from function import load_data
from nav_research import NavResearch
import pandas as pd
from pathlib import Path

basic_info = load_data("产品目录.xlsx")
# strategy_info = basic_info[basic_info["大类策略"] == "量化CTA"]


def multi_fund_comparison(tables, fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame()
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]
    for year in years:
        data3[f"{year}收益"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_收益"
        ].values
        data3[f"{year}最大回撤"] = data4.loc[
            data4["分年度业绩"] == year, f"{fund_name}_最大回撤"
        ].values
    data = pd.concat([data1, data2, data3], axis=1)
    data.drop(columns=["基准指数", "整体业绩"], inplace=True)
    return data


data = pd.DataFrame()
files_list_series = pd.Series(
    [i for i in Path("./data").rglob("*") if i.suffix.lower() in {".csv", ".xlsx",".xls"}]
)
for row in basic_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[3] in x.stem)]
    assert len(nav_df_path) == 1, "找到多个文件或者没有稳健"
    demo = NavResearch(nav_df_path.item(), row[0], row[3], row[4], row[5], row[6])
    demo.get_data()
    tables = demo.get_analysis_table()
    nav_df = multi_fund_comparison(tables, row[3])
    data = pd.concat([data, nav_df], axis=0)

data.to_excel("data.xlsx", index=False)